In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json("hf://datasets/selfrag/selfrag_train_data/train.jsonl", lines=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df['input'].head()

,input
0,
1,
2,
3,
4,


In [ ]:
!pip install sentence-transformers datasets pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from sentence_transformers import SentenceTransformer, util


In [ ]:
# Step 4: Define a financial context
financial_context = """
This text is related to finance, investments, stock markets,
financial instruments, banking, loans, and economic policies.
"""


In [ ]:
# Step 5: Load a Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # A lightweight and effective semantic model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
df['combined_text'] = df['instruction'].fillna('') + ' ' + df['output'].fillna('')



In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar

# Step 7: Compute embeddings with a loader
financial_embedding = model.encode(financial_context, convert_to_tensor=True)

# Apply model.encode with a loader over combined_text
text_embeddings = model.encode(
    list(tqdm(df['combined_text'], desc="Computing embeddings", unit="rows")),
    convert_to_tensor=True
)

Computing embeddings: 100%|██████████| 145619/145619 [00:00<00:00, 2349640.90rows/s]


In [ ]:
# Step 8: Calculate cosine similarity
cosine_scores = util.cos_sim(financial_embedding, text_embeddings).squeeze()

In [ ]:
# Step 9: Filter based on similarity threshold
threshold = 0.15  # Adjust threshold for precision
df['similarity_score'] = cosine_scores.tolist()
filtered_entries = df[df['similarity_score'] >= threshold]

In [ ]:
# Step 10: Save filtered entries
filtered_entries.to_csv("filtered_finance_entries.csv", index=False)


In [ ]:

# Display a preview of filtered entries
print(filtered_entries[['instruction', 'output', 'similarity_score']])

                                              instruction  \
1       Question: Write a text based on "rangers show ...   
4       Question: Fertility Clinics Vary on Embryo Dis...   
5       Teacher:In this task, you need to replace a le...   
7       Q: Write a positive tweet.\n\nA: Good night wo...   
13      Teacher:In this task, you are given a sentence...   
...                                                   ...   
145258  Given a chat history separated by new lines, g...   
145352  Given a chat history separated by new lines, g...   
145444  Given a chat history separated by new lines, g...   
145478  Given a chat history separated by new lines, g...   
145540  Given a chat history separated by new lines, g...   

                                                   output  similarity_score  
1       [No Retrieval]six opposition candidates in the...          0.247412  
4                  [No Retrieval]Answer: World[Utility:5]          0.173039  
5       [No Retrieval]a weird yel

In [ ]:
filtered_entries.describe()

,similarity_score
count,20.000000
mean,0.530982
std,0.032133
min,0.500433
25%,0.509831
50%,0.523841
75%,0.538376
max,0.621385


previous code

In [ ]:
finance_keywords = [
    'finance', 'investment', 'money', 'bank', 'economy',
    'financial', 'stocks', 'cryptocurrency', 'market', 'fund',
    'loan', 'interest', 'capital'
]

In [ ]:
finance_entries = df[
    df['instruction'].str.contains('|'.join(finance_keywords), case=False, na=False) |
    df['output'].str.contains('|'.join(finance_keywords), case=False, na=False)
]

In [ ]:
file_name = 'finance_entries.jsonl'
finance_entries.to_json(file_name, orient='records', lines=True)

In [ ]:
finance_entries['instruction']

,instruction
9,Fill in the response. A 2 person conversation:...
11,"If this is the response, what came before? Res..."
12,Title: DOA Review: We received the neat receip...
13,"Teacher:In this task, you are given a sentence..."
24,Imagine the conversation that came before this...
...,...
145610,"Given a chat history separated by new lines, g..."
145611,"Given a chat history separated by new lines, g..."
145613,"Given a chat history separated by new lines, g..."
145614,"Given a chat history separated by new lines, g..."


## Model

In [ ]:
!pip install unsloth

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel

tokenizer = AutoTokenizer.from_pretrained("selfrag/selfrag_llama2_7b")
model = AutoModelForCausalLM.from_pretrained("selfrag/selfrag_llama2_7b")
optimized_model = FastLanguageModel(model, tokenizer, use_fp16=True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


tokenizer_config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Dataset

In [ ]:
finrag = pd.read_parquet("hf://datasets/philschmid/finanical-rag-embedding-dataset/data/train-00000-of-00001.parquet")

In [ ]:
print(finrag.head())

In [ ]:
!wget https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz

--2024-12-15 01:04:56--  https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.128, 13.35.7.50, 13.35.7.82, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4694541059 (4.4G) [application/gzip]
Saving to: ‘psgs_w100.tsv.gz’

psgs_w100.tsv.gz    100%[===================>]   4.37G  36.1MB/s    in 2m 0s   

2024-12-15 01:06:57 (37.4 MB/s) - ‘psgs_w100.tsv.gz’ saved [4694541059/4694541059]



In [ ]:
import gzip
import pandas as pd

In [ ]:
file_path = 'psgs_w100.tsv.gz'

# Load the TSV file (tab-separated values)
with gzip.open(file_path, 'rt') as f:
    data = pd.read_csv(f, sep='\t', nrows=1000)

In [ ]:
print(data.head())

# Check the shape of the dataset (rows, columns)
print(data.shape)

# Display column names
print(data.columns)

   id                                               text  title
0   1  Aaron Aaron ( or ; "Ahärôn") is a prophet, hig...  Aaron
1   2  God at Sinai granted Aaron the priesthood for ...  Aaron
2   3  his rod turn into a snake. Then he stretched o...  Aaron
3   4  however, Aaron and Hur remained below to look ...  Aaron
4   5  Aaron and his sons to the priesthood, and arra...  Aaron
(1000, 3)
Index(['id', 'text', 'title'], dtype='object')


In [ ]:
print(data.title.unique())

['Aaron' 'Alcohol' 'Achill Island' 'Allen Ginsberg'
 'Algebraically closed field' 'Anatoly Karpov' 'Aspect ratio'
 'Auto racing' 'Anarcho-capitalism' 'Aristophanes' 'Albert Schweitzer'
 'Austrian School' 'Abscess' 'Aalborg Municipality' 'Aarhus'
 'Northern cavefish' 'Amateur' 'Alexis Carrel' "All Souls' Day"
 'Anatole France' 'André Gide' 'Algorithms for calculating variance'
 'Almond' 'Demographics of Antigua and Barbuda'
 'Politics of Antigua and Barbuda'
 'Telecommunications in Antigua and Barbuda'
 'Royal Antigua and Barbuda Defence Force' 'Antisemitism'
 'Economy of Azerbaijan' 'Geography of Azerbaijan'
 'Foreign relations of Azerbaijan']


In [ ]:
import pandas as pd

# Read CSV file
csv_file = "filtered_finance_entries_threshold_1_5.csv"  # Replace with your CSV file name
jsonl_file = "output.jsonl"

# Load the CSV data into a DataFrame
df = pd.read_csv(csv_file)

# Save DataFrame to JSONL
df.to_json(jsonl_file, orient="records", lines=True)

print(f"CSV file successfully converted to {jsonl_file}")

CSV file successfully converted to output.jsonl


In [ ]:
# View first few lines of a JSONL file
jsonl_file = "output.jsonl"

# Open the file and read the first few lines
num_lines = 5  # Number of lines to display

with open(jsonl_file, "r") as file:
    for i, line in enumerate(file):
        if i >= num_lines:
            break
        print(line.strip())

FileNotFoundError: [Errno 2] No such file or directory: 'output.jsonl'

In [ ]:
# View first few lines of a JSONL file
jsonl_file = "output.jsonl"

# Open the file and read the first few lines
num_lines = 5  # Number of lines to display

with open(jsonl_file, "r") as file:
    for i, line in enumerate(file):
        if i >= num_lines:
            break
        print(line.strip())

{"instruction":"Question: Write a text based on \"rangers show some iron to tie the series\"\nText: along with being talented , self-assured and highly paid , these experienced rangers are also considerate .\n\nQuestion: Write a text based on \"union wo n't dismantle blockage of gm canada headquarters\"\nText: canadian auto workers officials friday refused to end a blockade of general motors canada headquarters despite an offer to potentially bring new car production to a complex where a truck plant is slated for closure .\n\nQuestion: Write a text based on \"six azerbaijan opposition parliamentary candidates declare hunger strike\"\nText:","output":"[No Retrieval]six opposition candidates in the upcoming parliamentary elections in azerbaijan declared a hunger strike friday to pressure the government into ensuring fair polls .[Utility:5]","input":null,"id":"flan_v2_18667","dataset_name":"flan_v2","combined_text":"Question: Write a text based on \"rangers show some iron to tie the serie